# Data Extraction 

In [ ]:
import fastf1
import pandas as pd 

In [ ]:
#Enable cache 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")

In [ ]:
#Define the gps by year of interest 
gps_by_year = {
    2019: ["Bahrain", "China", "Australia", "Japan", "Monaco", "Spain", "Canada", 
           "Austria", "Great Britain", "Hungary", "Belgium", "Azerbaijan", 
           "Singapore", "Austin", "Mexico", "Brazil", "Abu Dhabi", "Monza",
           "France", "Germany","Russia"],

    2020: ["Bahrain", "Spain", "Austria", "Great Britain", "Hungary", "Belgium", 
           "Abu Dhabi", "Monza", "Turkey", "Portugal", "Russia", "Imola", 
           "Eifel", "Mugello", "Sakhir","70th","Sakhir"],

    2021: ["Bahrain", "Monaco", "Spain", "Imola", "Austria", 
           "Great Britain", "Hungary", "Belgium", "Azerbaijan", "Austin", 
           "Mexico", "Brazil", "Abu Dhabi", "Monza", "Qatar", "Saudi Arabia",
           "France", "Netherlands", "Turkey", "Russia","Portugal","Styria"],

    2022: ["Bahrain", "Australia", "Japan", "Monaco", "Spain", 
           "Canada", "Austria", "Great Britain", "Hungary", 
           "Belgium", "Azerbaijan", "Singapore", "Austin", "Mexico", 
           "Brazil", "Abu Dhabi", "Monza", "Qatar", "Miami", "Saudi Arabia",
           "France", "Netherlands"],

    2023: ["Bahrain", "Saudi Arabia", "Australia", "Japan", 
           "Miami", "Monaco", "Spain", "Canada", "Austria", 
           "Great Britain", "Hungary", "Belgium", "Netherlands", 
           "Azerbaijan", "Singapore", "Austin", "Mexico", "Brazil", 
           "Qatar", "Abu Dhabi", "Monza", "Las Vegas"],

    2024: ["Bahrain", "Saudi Arabia", "Australia", "Japan", 
           "China", "Miami", "Monaco", "Spain", "Canada", "Austria", 
           "Great Britain", "Hungary", "Belgium", "Netherlands", "Monza"]
}

## Qualifying data

Obtain Q1 and Q2 data, this is the focus of the project

Running and treatment variables 

In [ ]:
#Q1 and Q2 data  
q1_combined = pd.DataFrame(columns=["Driver", "Team", "LapTime", "Position", "GapToKnockout", "Session", "Year", "GP"])
q2_combined = pd.DataFrame(columns=["Driver", "Team", "LapTime", "Position", "GapToKnockout","Session", "Year", "GP"])


for year, gps in gps_by_year.items():
    
    for gp in gps:
        try:
            # Load the session data
            session = fastf1.get_session(year, gp, 'Q')
            session.load()

            # Extract relevant results
            results = session.results[["Abbreviation", "TeamName", "Q1", "Q2"]]
            results["Q1"] = results["Q1"].dt.total_seconds()
            results["Q2"] = results["Q2"].dt.total_seconds()

            # Process Q1 data
            q1 = results[["Abbreviation", "TeamName", "Q1"]].sort_values(by="Q1").reset_index(drop=True)
            q1.columns = ["Driver", "Team", "LapTime"]
            q1["Position"] = q1.index + 1
            knockout_time_q1 = q1.iloc[14]["LapTime"]  # 15th place time
            q1["GapToKnockout"] = q1["LapTime"] - knockout_time_q1
            q1["Year"] = year
            q1["GP"] = gp
            q1["Session"] = "Q1"
            q1_combined = pd.concat([q1_combined, q1], ignore_index=True)

            # Process Q2 data
            q2 = results[["Abbreviation", "TeamName", "Q2"]].dropna().sort_values(by="Q2").reset_index(drop=True)
            q2.columns = ["Driver", "Team", "LapTime"]
            q2["PositionQ2"] = q2.index + 1
            knockout_time_q2 = q2.iloc[9]["LapTime"]  # 10th place time
            q2["GapToKnockoutQ2"] = q2["LapTimeQ2"] - knockout_time_q2
            q2["Year"] = year
            q2["GP"] = gp
            q2["Session"] = "Q2"
            q2_combined = pd.concat([q2_combined, q2], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")


q1_combined.to_csv("PaperQ1Data.csv", index=False)
q2_combined.to_csv("PaperQ2Data.csv", index=False)

## Race Data

Obtain data from Race Day! 

Outcomes and covariates

In [ ]:
#Race Data
race_combined = pd.DataFrame(columns=["Driver", "Team", "GridPosition", "RacePosition", "Status", 
                                      "Points", "PositionDifference", "Year", "GP", "ChampionshipPoints", 
                                      "LastRacePosition", "TeamChampionshipPoints", "TeammateLastRacePosition"])

# Loop through each year
for year, gps in gps_by_year.items():

    # Initialize dictionaries to keep track of championship standings
    driver_points = {}
    team_points = {}
    last_race_positions = {}

    
    for gp in gps:
        try:
            # Load the race session data
            session = fastf1.get_session(year, gp, 'R')
            session.load()

            # Extract relevant race results
            results = session.results[["Abbreviation", "TeamName", "GridPosition", "Position", "Status", "Points"]]
            results["PositionDifference"] = results["GridPosition"] - results["Position"]

            # Update driver and team points
            for _, row in results.iterrows():
                driver = row["Abbreviation"]
                team = row["TeamName"]
                points = row["Points"]
                
                if driver not in driver_points:
                    driver_points[driver] = 0
                if team not in team_points:
                    team_points[team] = 0

                driver_points[driver] += points
                team_points[team] += points

                # Last race position for the driver
                last_race_positions[driver] = row["Position"]

            # Prepare the data for saving
            race_data = []
            for _, row in results.iterrows():
                driver = row["Abbreviation"]
                team = row["TeamName"]
                
                # Get points and positions
                championship_points = driver_points[driver]
                last_race_position = last_race_positions.get(driver, np.nan)
                team_championship_points = team_points.get(team, np.nan)
                
                # Find teammate's last race position
                teammates = results[results["TeamName"] == team]["Abbreviation"].tolist()
                teammates.remove(driver)
                teammate_last_race_position = last_race_positions.get(teammates[0], np.nan) if teammates else np.nan

                race_data.append({
                    "Driver": driver,
                    "Team": team,
                    "GridPosition": row["GridPosition"],
                    "RacePosition": row["Position"],
                    "Status": row["Status"],
                    "Points": row["Points"],
                    "PositionDifference": row["PositionDifference"],
                    "Year": year,
                    "GP": gp,
                    "ChampionshipPoints": championship_points,
                    "LastRacePosition": last_race_position,
                    "TeamChampionshipPoints": team_championship_points,
                    "TeammateLastRacePosition": teammate_last_race_position
                })

            # Combine the data
            race_combined = pd.concat([race_combined, pd.DataFrame(race_data)], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")


race_combined.to_csv("PaperRaceData.csv", index=False)

## Free Practices Data

Covariates 

In [ ]:
#FP1 data

fp1_combined = pd.DataFrame(columns=["Driver", "Team", "LapTime", "GapToFastestFP1", "Position", "Year", "GP", "Session"])


for year, gps in gps_by_year.items():
    for gp in gps:
        try:
            # Load the FP1 session data
            session = fastf1.get_session(year, gp, 'FP1')
            session.load()

            # Get all laps in the session
            laps = session.laps

            # Convert LapTime to total seconds
            laps['LapTime'] = laps['LapTime'].dt.total_seconds()

            # Drop rows with NaN LapTimes (e.g., if a driver did not set a valid lap time)
            laps = laps.dropna(subset=['LapTime'])

            # Get the fastest lap per driver
            fastest_laps = laps.groupby('Driver').apply(lambda x: x.nsmallest(1, 'LapTime')).reset_index(drop=True)

            # Find the fastest lap time in the session
            session_fastest_time = fastest_laps["LapTime"].min()

            # Calculate the gap to the session's fastest lap for each driver's fastest lap
            fastest_laps["GapToFastestFP1"] = fastest_laps["LapTime"] - session_fastest_time

            # Extract the necessary columns and rename them
            laps_data = fastest_laps[["Driver", "Team", "LapTime", "GapToFastestFP1"]]
            laps_data["Position"] = laps_data["LapTime"].rank().astype(int)  # Rank laps by time
            laps_data["Year"] = year
            laps_data["GP"] = gp
            laps_data["Session"] = "FP1"

            # Combine the data
            fp1_combined = pd.concat([fp1_combined, laps_data], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")


fp1_combined.to_csv("PaperFP1Data.csv", index=False)

In [ ]:
#FP2 data
fp2_combined = pd.DataFrame(columns=["Driver", "Team", "LapTime", "GapToFastestFP2", "Position", "Year", "GP", "Session"])


for year, gps in gps_by_year.items():
    for gp in gps:
        try:
            # Load the FP2 session data
            session = fastf1.get_session(year, gp, 'FP2')
            session.load()

            # Get all laps in the session
            laps = session.laps

            # Convert LapTime to total seconds
            laps['LapTime'] = laps['LapTime'].dt.total_seconds()

            # Drop rows with NaN LapTimes (e.g., if a driver did not set a valid lap time)
            laps = laps.dropna(subset=['LapTime'])

            # Get the fastest lap per driver
            fastest_laps = laps.groupby('Driver').apply(lambda x: x.nsmallest(1, 'LapTime')).reset_index(drop=True)

            # Find the fastest lap time in the session
            session_fastest_time = fastest_laps["LapTime"].min()

            # Calculate the gap to the session's fastest lap for each driver's fastest lap
            fastest_laps["GapToFastestFP2"] = fastest_laps["LapTime"] - session_fastest_time

            # Extract the necessary columns and rename them
            laps_data = fastest_laps[["Driver", "Team", "LapTime", "GapToFastestFP2"]]
            laps_data["Position"] = laps_data["LapTime"].rank().astype(int)  # Rank laps by time
            laps_data["Year"] = year
            laps_data["GP"] = gp
            laps_data["Session"] = "FP2"

            # Combine the data
            fp2_combined = pd.concat([fp1_combined, laps_data], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")

# Save the combined data for all years to a single CSV file
fp2_combined.to_csv("PaperFP2Data.csv", index=False)

In [ ]:
#FP3 Data
fp3_combined = pd.DataFrame(columns=["Driver", "Team", "LapTime", "GapToFastestFP3", "Position", "Year", "GP", "Session"])


for year, gps in gps_by_year.items():
    
    for gp in gps:
        try:
            # Load the FP3 session data
            session = fastf1.get_session(year, gp, 'FP3')
            session.load()

            # Get all laps in the session
            laps = session.laps

            # Convert LapTime to total seconds
            laps['LapTime'] = laps['LapTime'].dt.total_seconds()

            # Drop rows with NaN LapTimes (e.g., if a driver did not set a valid lap time)
            laps = laps.dropna(subset=['LapTime'])

            # Get the fastest lap per driver
            fastest_laps = laps.groupby('Driver').apply(lambda x: x.nsmallest(1, 'LapTime')).reset_index(drop=True)

            # Find the fastest lap time in the session
            session_fastest_time = fastest_laps["LapTime"].min()

            # Calculate the gap to the session's fastest lap for each driver's fastest lap
            fastest_laps["GapToFastestFP3"] = fastest_laps["LapTime"] - session_fastest_time

            # Extract the necessary columns and rename them
            laps_data = fastest_laps[["Driver", "Team", "LapTime", "GapToFastestFP3"]]
            laps_data["Position"] = laps_data["LapTime"].rank().astype(int)  # Rank laps by time
            laps_data["Year"] = year
            laps_data["GP"] = gp
            laps_data["Session"] = "FP3"

            # Combine the data
            fp3_combined = pd.concat([fp1_combined, laps_data], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")


fp3_combined.to_csv("PaperFP3Data.csv", index=False)

## Pitstop Data 

Outcomes

In [ ]:
#Pitstop data
pit_stop_data = pd.DataFrame(columns=["Driver", "Team", "Year", "GP", "StartingTyre", 
                                      "NumberOfPitStops", "TyresUsed"])

# Loop through each year
for year, gps in gps_by_year.items():
    # Loop through each GP for the current year
    for gp in gps:
        try:
            # Load the race session data
            session = fastf1.get_session(year, gp, 'R')
            session.load()

            # Extract lap data
            laps = session.laps

            # Initialize variables for each driver
            driver_starting_tire = {}
            driver_pit_stops = {}
            driver_tires_used = {}

            # Convert PitInTime to seconds
            laps['PitOutTime'] = laps['PitOutTime'].dt.total_seconds()

            # Filter laps to identify pit stops
            pit_stops = laps[laps['PitOutTime'] > 0]

            # Ensure we are not counting the first lap out of the pits
            valid_pit_stops = pit_stops[pit_stops['LapNumber'] > 1]

            # Loop through each lap to determine tire changes and pit stops
            for _, lap in laps.iterrows():
                driver = lap['Driver']
                compound = lap['Compound']
                
                if driver not in driver_starting_tire:
                    driver_starting_tire[driver] = compound
                
                if driver not in driver_pit_stops:
                    driver_pit_stops[driver] = 0
                
                if driver not in driver_tires_used:
                    driver_tires_used[driver] = []

                # Append the tire compound to the list for this driver
                if not driver_tires_used[driver] or driver_tires_used[driver][-1] != compound:
                    driver_tires_used[driver].append(compound)

            # Count the number of pit stops
            for _, pit in valid_pit_stops.iterrows():
                driver = pit['Driver']
                driver_pit_stops[driver] += 1

            # Prepare the data for saving
            race_data = []
            for driver in driver_starting_tire.keys():
                race_data.append({
                    "Driver": driver,
                    "Team": session.results.loc[session.results["Abbreviation"] == driver, "TeamName"].values[0],
                    "Year": year,
                    "GP": gp,
                    "StartingTire": driver_starting_tire[driver],
                    "NumberOfPitStops": driver_pit_stops.get(driver, 0),
                    "TiresUsed": ', '.join(driver_tires_used.get(driver, []))
                })

            # Combine the data
            pit_stop_data = pd.concat([pit_stop_data, pd.DataFrame(race_data)], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {gp} {year}: {e}")

# Save the pit stop data to a CSV file for verification
pit_stop_data.to_csv("PitStopData.csv", index=False)

## Strategy Data 

Outcomes

In [ ]:
#Strategy data
all_strategy_data = []

# Loop through each year and each GP
for year, gps in gps_by_year.items():
    for gp in gps:
        try:
            # Load the race session for each GP and year 
            race = fastf1.get_session(year, gp, 'R')
            race.load()

            # Get the laps data for the race
            laps = race.laps
            drivers = race.drivers
            drivers = [race.get_driver(driver)["Abbreviation"] for driver in drivers]

            # Extracting strategy information for each driver
            for driver in drivers:
                # Get driver details
                driver_info = race.get_driver(driver)
                team = driver_info["TeamName"]
                
                # Filter laps for the current driver
                driver_laps = laps[laps["Driver"] == driver]

                # Create a strategy string capturing compounds for each outlap
                strategy = []

                # Initialize the strategy with the starting compound
                starting_compound = driver_laps['Compound'].iloc[0]
                
                strategy.append(starting_compound)

                # Flag to skip the first outlap
                first_outlap_skipped = False
                for i in range(len(driver_laps)):
                    # Check if the current lap is an outlap based on PitOutTime
                    if driver_laps['PitOutTime'].dt.total_seconds().iloc[i] > 0:
                        # Skip the first outlap
                        if not first_outlap_skipped:
                            first_outlap_skipped = True
                            continue
                        
                        # Capture the compound used on this outlap
                        outlap_compound = driver_laps['Compound'].iloc[i]
                        strategy.append(outlap_compound)

                # Join the strategy components with a '-'
                strategy_string = '-'.join(strategy)

                # Append the strategy data for this driver, GP, year, and team
                all_strategy_data.append({
                    'Driver': driver,
                    'Team': team,
                    'GP': gp, 
                    'Year': year,  
                    'Strategy': strategy_string
                })

        except Exception as e:
            print(f"Error loading {gp} {year}: {e}")

# Convert the list of strategy data into a DataFrame
strategy_df = pd.DataFrame(all_strategy_data)

#Team mapping
team_mapping = {
    'Toro Rosso': 'RB',
    'AlphaTauri':'RB',
    'Renault': 'Alpine',
    'Alfa Romeo Racing': 'Kick Sauber',
    'Alfa Romeo': 'Kick Sauber',
    'Racing Point': 'Aston Martin'
}

# Replace team names in the 'Team' column using the mapping
strategy_df['Team'] = strategy_df['Team'].replace(team_mapping)

strategy_df.to_csv("Strategy.csv")

# Merge and additional variables construction 

Merge all the intermediate datasets to obtain the final dataset 

In [ ]:
#Merge al the datasets 

# Load all the DataFrames
q1 = pd.read_csv('PaperQ1Data.csv')    
q2 = pd.read_csv('PaperQ2Data.csv')   
race = pd.read_csv('PaperRaceData.csv')  
FP1 = pd.read_csv('PaperFP1Data.csv')   
FP2 = pd.read_csv('PaperFP2Data.csv')   
FP3 = pd.read_csv('PaperFP3Data.csv')   
pitstop=pd.read_csv('PitStopData.csv') 


q1 = q1.rename(columns={"GapToKnockout": "GapToKnockoutQ1"})
q1 = q1.rename(columns={"LapTime": "LapTimeQ1"})
q1 = q1.rename(columns={"Position": "PositionQ1"})
q1 = q1.drop('Session', axis=1)

q2= q2.rename(columns={"GapToKnockout": "GapToKnockoutQ2"})
q2=q2.rename(columns={"LapTime": "LapTimeQ2"})
q2 = q2.rename(columns={"Position": "PositionQ2"})
q2 = q2.drop('Session', axis=1)

FP1=FP1.rename(columns={"LapTime": "LapTimeFP1"})
FP1=FP1.rename(columns={"Position": "PositionFP1"})
FP1 = FP1.drop('Session', axis=1)

FP2=FP2.rename(columns={"LapTime": "LapTimeFP2"})
FP2=FP2.rename(columns={"Position": "PositionFP2"})
FP2 = FP2.drop('Session', axis=1)

FP3=FP3.rename(columns={"LapTime": "LapTimeFP3"})
FP3=FP3.rename(columns={"Position": "PositionFP3"})
FP3 = FP3.drop('Session', axis=1)


# Start by merging q1 and q2
merged_df = pd.merge(q1, q2, on=['GP', 'Year', 'Driver','Team'], how='left')

# Merge the result with race data
merged_df = pd.merge(merged_df, race, on=['GP', 'Year', 'Driver','Team'], how='left')

# Merge with FP1
merged_df = pd.merge(merged_df, FP1, on=['GP', 'Year', 'Driver','Team'], how='left')

# Merge with FP2
merged_df = pd.merge(merged_df, FP2, on=['GP', 'Year', 'Driver','Team'], how='left')

# Merge with FP3
merged_df = pd.merge(merged_df, FP3, on=['GP', 'Year', 'Driver','Team'], how='left')

merged_df = pd.merge(merged_df, pitstop, on=['GP', 'Year', 'Driver','Team'], how='left')




In [ ]:
#Create Treatment Variables, Dummies 

#Treatment 
# Create a new variable to indicate if the driver made it to Q2
merged_df['MadeItToQ2'] = (merged_df['GapToKnockoutQ1'] <= 0).astype(int)

# Create a new variable to indicate if the driver made it to Q3
merged_df['MadeItToQ3'] = (merged_df['GapToKnockoutQ2'] <= 0).astype(int)

#Starting Tyre Dummies
dummies = merged_df.get_dummies(merged_df['StartingTyre'], prefix='Tyre', prefix_sep='').astype(int)
merged_df = merged_df.concat([merged_df, dummies], axis=1)

#Dummy for Pitlane Start
merged_df['Pitlane'] = (merged_df['GridPosition'] == 0).astype(int)

#DNF Type Dummies 

dnf_driving = [
    'Collision damage', 'Collision', 'Spun off', 'Accident', 'Damage'
]

dnf_mechanical = [
    'Engine', 'Power Unit', 'Transmission', 'Oil leak', 'Fuel leak', 'Fuel pump', 
    'Exhaust', 'Hydraulics', 'Electronics', 'Fuel pressure', 'Gearbox', 'Mechanical', 
    'Undertray', 'Technical', 'Brakes', 'Suspension', 'Wheel', 'Power loss', 'Puncture', 
    'Radiator', 'Wheel nut', 'Driveshaft', 'Electrical', 'Turbo', 'Rear wing', 'Vibrations', 
    'Water leak', 'Front wing', 'Cooling system', 'Water pump', 'Differential', 'Steering', 
    'Debris'
]

merged_df['DNFDriving'] = merged_df['Status'].isin(dnf_driving).astype(int)


merged_df['DNFMechanical'] = merged_df['Status'].isin(dnf_mechanical).astype(int)


In [ ]:
#Save final dataset to csv
merged_df.to_csv("FinalDataSet.csv")